In [1]:
import pybamm
import matplotlib.pyplot as plt
from basic_spm_msmr import BasicSPMSMR
import numpy as np

In [2]:
model = BasicSPMSMR()
parameter_values = model.default_parameter_values

In [4]:
%matplotlib

x_n = parameter_values["Negative electrode stoichiometry"]
x_p = parameter_values["Positive electrode stoichiometry"]

fig, ax = plt.subplots(2, 2, figsize=(10, 4))

U = pybamm.linspace(0.01, 1.5, 500)
x_eval = parameter_values.evaluate(x_n(U)).flatten()
U_eval = U.evaluate().flatten()
dUdx_eval = -np.gradient(U_eval, x_eval)
ax[0, 0].plot(U_eval, x_eval, label="x_n")
ax[0, 0].set_xlabel("U_n")
ax[0, 0].set_ylabel("x_n")
ax[1, 0].plot(x_eval, dUdx_eval, label="x_n")
ax[1, 0].set_xlabel("x_n")
ax[1, 0].set_ylabel("dU_n/dx_n")
ax[1, 0].set_ylim([0, 5])

U = pybamm.linspace(3.4, 5, 500)
x_eval = parameter_values.evaluate(x_p(U)).flatten()
U_eval = U.evaluate().flatten()
dUdx_eval = -np.gradient(U_eval, x_eval)
ax[0, 1].plot(U_eval, x_eval, label="x_p")
ax[0, 1].set_xlabel("U_p")
ax[0, 1].set_ylabel("x_p")
ax[1, 1].plot(x_eval, dUdx_eval, label="x_p")
ax[1, 1].set_xlabel("x_p")
ax[1, 1].set_ylabel("dU_p/dx_p")
ax[1, 1].set_ylim([0, 5])


Using matplotlib backend: MacOSX


(0.0, 5.0)

2023-05-31 15:10:33.725 Python[90554:1895843] *** Assertion failure in +[NSEvent otherEventWithType:location:modifierFlags:timestamp:windowNumber:context:subtype:data1:data2:], NSEvent.m:647
2023-05-31 15:10:33.738 Python[90554:1895843] *** Assertion failure in +[NSEvent otherEventWithType:location:modifierFlags:timestamp:windowNumber:context:subtype:data1:data2:], NSEvent.m:647
2023-05-31 15:10:33.750 Python[90554:1895843] *** Assertion failure in +[NSEvent otherEventWithType:location:modifierFlags:timestamp:windowNumber:context:subtype:data1:data2:], NSEvent.m:647
2023-05-31 15:10:33.763 Python[90554:1895843] *** Assertion failure in +[NSEvent otherEventWithType:location:modifierFlags:timestamp:windowNumber:context:subtype:data1:data2:], NSEvent.m:647
2023-05-31 15:10:33.775 Python[90554:1895843] *** Assertion failure in +[NSEvent otherEventWithType:location:modifierFlags:timestamp:windowNumber:context:subtype:data1:data2:], NSEvent.m:647
2023-05-31 15:10:33.787 Python[90554:1895843]

: 

In [1]:
soc_model = pybamm.BaseModel()
U_n = pybamm.Variable("U_n")
U_p = pybamm.Variable("U_p")
soc_model.variables = {"U_n": U_n, "U_p": U_p}
x_0 = parameter_values["Negative electrode stoichiometry at 0% SOC"]
x_100 = parameter_values["Negative electrode stoichiometry at 100% SOC"]
y_0 = parameter_values["Positive electrode stoichiometry at 0% SOC"]
y_100 = parameter_values["Positive electrode stoichiometry at 100% SOC"]
initial_soc = pybamm.InputParameter("Initial soc")
x = x_0 + initial_soc * (x_100 - x_0)
y = y_0 - initial_soc * (y_0 - y_100)
soc_model.algebraic = {U_n: x - x_n(U_n), U_p: y - x_p(U_p)}
soc_model.initial_conditions = {U_n: pybamm.Scalar(0), U_p: pybamm.Scalar(4)}
parameter_values.process_model(soc_model)
soc_sol = pybamm.AlgebraicSolver(tol=1e-6).solve(soc_model, inputs={"Initial soc": 1})
U_n, U_p = soc_sol["U_n"].data[0], soc_sol["U_p"].data[0]

parameter_values.update(
    {
        "Initial negative electrode potential [V]": U_n,
        "Initial positive electrode potential [V]": U_p,
    },
    check_already_exists=False,
)
print(U_n, U_p)

NameError: name 'pybamm' is not defined

In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3300])
sim.plot()